In [2]:
import pandas as pd
import numpy as np
import re
from pathlib import Path
data_location = Path("../Data/Tableau_Terra_workspace")

# Create the terra workspace 
Read all these weird files and try to merge them to the best of my capabilities:

1. Read and merge CLF with Premium
2. Read and merge Panel data
3. Read RNA data
4. Read and merge WES data - UPCOMMING
5. Apply loop through rows
6. Duplicate rows and add conditions
7. Check number of datapoints

Note, losing alot of information due to inconsistencies in the datasets. Another note this workflow eats up memory, because I keep meta data open, just something to note

### 1 Read and merge CLF with Premium

In [3]:
#CLF data and cleaning
CLF_data_meta = pd.read_csv(data_location / "CLF_Cohort_Summary_full_data.csv", encoding = 'utf-8', sep='\t')
CLF_data = CLF_data_meta[["Cell Line", "PT-ID", "SM-ID", "Status", "Resolution", "T/N", "Primary Disease", "Tissue site", "Tumor type", "CELL_LINE_ID(raw)", "Media type", "Growth Properties", "Flask coating", "Starting Material Type", "Fail Mode"]]
CLF_data = CLF_data.replace(r'^\s*$', np.nan, regex=True)
CLF_data = CLF_data.rename(columns={"Cell Line": "Cell Line Starting Sample", "Tissue site": "Tissue Site", "Tumor type": "Tumor Type", "CELL_LINE_ID(raw)":"Cell Line", "Growth Properties": "Dimension"})
CLF_data["Dataset"] = "CLF"

In [4]:
#Premium data and cleaning
Premium_data_meta = pd.read_csv(data_location / "Premium_Cohort_Summary_full_data.csv", encoding = 'utf-8', sep='\t')
Premium_data = Premium_data_meta[["Cell Line Starting Sample", "PT_ID", "Passage Sample", "JIRA Status", "Resolution", "T/N", "Primary disease", "Tissue Site", "Tumor Type", "CELL_LINE_ID", "Growth Medium", "Growth Pattern", "ECM Bio Coatings", "Root Material Type", "Passage Sample Status"]]
Premium_data = Premium_data.replace(r'^\s*$', np.nan, regex=True)
Premium_data = Premium_data.rename(columns={"PT_ID": "PT-ID", "Passage Sample": "SM-ID", "JIRA Status": "Status", "Primary disease": "Primary Disease", "CELL_LINE_ID":"Cell Line", "Growth Medium":"Media type", "Growth Pattern":"Dimension", "ECM Bio Coatings":"Flask coating", "Root Material Type":"Starting Material Type", "Passage Sample Status":"Fail Mode"})
Premium_data["Dataset"] = "Premium"

/opt/miniconda3/envs/Broad_env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3062: DtypeWarning: Columns (5,33,80,95) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [5]:
# Merge CLF and Premium
data = pd.concat([CLF_data, Premium_data])

# Remove Normals
data = data[data["T/N"] == "Tumor"]

# Cleanup some of columns
data["SM-ID"] = [re.split('\r|  ',x)[0].strip(' \t\n\r') if str(x) != 'nan' else x for x in list(data["SM-ID"])]
data["Resolution"] = [str(x) for x in list(data["Resolution"])]
data["Cell Line"] = [str(x) for x in list(data["Cell Line"])]
data["Cell Line"] = [re.split('\r|  ',x)[0].strip(' \t\n\r') if str(x) != 'nan' else x for x in list(data["Cell Line"])]
data["Media type"] = [str(x) for x in list(data["Media type"])]
data = data[data["Media type"] != "nan"]

# Note Dropping lot of conditions
Resolution_dict = {
    'Terminated':'Terminated',
    'In Process':'In Process',
    'Verification':'In Process',
    'Ambiguous':'Drop',
    'Completed':'Completed',
    'Verified Normal':'Verified Normal',
    'Verified Tumor':'Verified Tumor',
    'Clinical team follow up':'In Process',
    'Other verified tumor models':'Verified Tumor',
    'Primary sample verified normal':'Verified Normal',
    'Blacklisted; FP Failure':'Drop',
    'Low Purity Tumor':'Drop',
    'Clinical team follow up,Primary sample verified normal':'Verified Normal',
    'Clinical team follow up,Other verified tumor models':'Verified Tumor',
    'nan':'nan',
    'Abandoned':'Drop',
}

data["Resolution_modified"] = data["Resolution"]
data = data.replace({"Resolution_modified": Resolution_dict})
data = data[(data["Resolution_modified"] != "Drop") & (data["Resolution_modified"] != "In Process")]

### 2 Read and combined Panel data

In [6]:
# TWIST Panel data
TWIST_samples_meta = pd.read_csv(data_location / "PANCAN_TWIST_sample.tsv", sep="\t")
TWIST_pair_meta = pd.read_csv(data_location / "PANCAN_TWIST_pair.tsv", sep="\t")
TWIST_samples = TWIST_samples_meta[["entity:sample_id", "external_id_validation", "participant", "patient_id", "sample_type", "renamed_bai_file", "renamed_bam_file", "depth_of_cov_qc_result", "cnv_calls", "media"]]
TWIST_samples['tn_decision_clean'] = "Not Reported"
TWIST_pair = TWIST_pair_meta[["case_sample", "filtered_variants", "mutect1_callstats", "mutect1_coveragewig", "mutect1_powerwig", "mutect1_vcf", "mutect2_vcf", "oncotated_maf_mutect1", "oncotated_maf_mutect2"]]

TWIST_samples = TWIST_samples[TWIST_samples["entity:sample_id"].isin(TWIST_pair["case_sample"])]
TWIST_samples = TWIST_samples.rename(columns={"entity:sample_id":"case_sample"})
TWIST = pd.merge(TWIST_samples, TWIST_pair, on='case_sample')
TWIST = TWIST.replace(r'^\s*$', np.nan, regex=True)
TWIST["Dataset"] = "TWIST"

<ipython-input-6-dde66bea5383>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  TWIST_samples['tn_decision_clean'] = "Not Reported"


In [7]:
# TSCA Panel data
TSCA_samples_meta = pd.read_csv(data_location / "TSCA_HCMI_sample.tsv", sep="\t")
TSCA_pair_meta = pd.read_csv(data_location / "TSCA_HCMI_pair.tsv", sep="\t")
TSCA_samples = TSCA_samples_meta[["entity:sample_id", "bsp_sample_id_validation", "external_id_validation", "participant", "sample_type", "renamed_bai_file", "renamed_bam_file", "depth_of_cov_qc_result", "cnv_calls", "media", "tn_decision_clean"]]
TSCA_pair = TSCA_pair_meta[["case_sample", "filtered_variants", "mutect1_callstats", "mutect1_coveragewig", "mutect1_powerwig", "mutect1_vcf", "mutect2_vcf", "oncotated_maf_mutect1", "oncotated_maf_mutect2"]]
TSCA_pair = TSCA_pair.dropna()

TSCA_samples = TSCA_samples[TSCA_samples["entity:sample_id"].isin(TSCA_pair["case_sample"])]
TSCA_samples = TSCA_samples.rename(columns={"entity:sample_id":"case_sample"})
TSCA = pd.merge(TSCA_samples, TSCA_pair, on='case_sample')
TSCA = TSCA.drop(columns=["case_sample"])
TSCA = TSCA.rename(columns={"bsp_sample_id_validation":"case_sample"})
TSCA = TSCA.replace(r'^\s*$', np.nan, regex=True)
TSCA["Dataset"] = "TSCA"

In [8]:
# Merge TWIST and TSCA with Tableau_data
Panel = pd.concat([TWIST, TSCA])

#Adding PANEL column head
Panel = Panel.rename(columns={"patient_id":"PT-ID"})
Panel.columns = ["PANEL_" + x if i > 4 else x for i,x in enumerate(Panel.columns)]

Panel["participant"] = [str(x) for x in Panel["participant"]]
Panel["PANEL_tn_decision_clean"] = [str(x) for x in Panel["PANEL_tn_decision_clean"]]

### 3 Read RNA data

In [9]:
# RNA data
RNA_meta = pd.read_csv(data_location / "CCLF_RNA_2_0.tsv", sep="\t")
RNA = RNA_meta[["entity:sample_id", 
                     "bsp_sample_id_rna",  
                     "participant", 
                     "sample_type", 
                     "fastq1", 
                     "fastq2", 
                     "bai_file_rna", 
                     "bam_file_rna", 
                     "fusion_predictions", 
                     "fusion_predictions_abridged", 
                     "rnaseqc_count_metrics", 
                     "rnaseqc_count_outputs", 
                     "rnaseqc_exon_counts", 
                     "rnaseqc_gene_counts", 
                     "rnaseqc_gene_rpkm", 
                     "rsem_genes", 
                     "rsem_isoforms", 
                     "star_bam_file", 
                     "star_bam_index", 
                     "star_chimeric_bam_index", 
                     "star_chimeric_junctions", 
                     "star_junctions", 
                     "star_junctions_pass1", 
                     "star_md_bam_file", 
                     "star_md_bam_index", 
                     "star_read_counts",
                    "star_transcriptome_bam"]]

In [10]:
#Adding RNA column head
RNA.columns = ["RNA_" + x if i > 3 else x for i,x in enumerate(RNA.columns)]

In [11]:
# Removing without SM-ID
total_rows = len(RNA)
RNA["bsp_sample_id_rna"] = [str(x) for x in RNA["bsp_sample_id_rna"]]
RNA["participant"] = [str(x) for x in RNA["participant"]]

<ipython-input-11-0eccdc508b84>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  RNA["bsp_sample_id_rna"] = [str(x) for x in RNA["bsp_sample_id_rna"]]
<ipython-input-11-0eccdc508b84>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  RNA["participant"] = [str(x) for x in RNA["participant"]]


### 4 Read WES data

In [12]:
# WES data
WES_samples_meta = pd.read_csv(data_location / "CCLF_WES_sample.tsv", sep="\t")
#WES_pair_meta = pd.read_csv(data_location / "CCLF_WES_pair.tsv", sep="\t")
WES_samples = WES_samples_meta[["entity:sample_id", "bsp_sample_id_capture", "external_id_capture", "participant", "sample_type", "formatted_bai_file", "formatted_bam_file", "cnv_calls"]]

# Pair is wierd

In [13]:
WES = WES_samples
WES.columns = ["WES_" + x if i > 4 else x for i,x in enumerate(WES.columns)]

### 5 Loop through data samples

In [14]:
def growth_check(x):
    mapping_dict = {
        'Terminated No Growth': 0,
        'Terminated Unlisted': 0,
        'Terminated Fungus Contamination': 0,
        'Terminated Stopped Growing': 0,
        'Terminated Fungus Contamination,No Growth': 0,
        'Terminated Bacterial Contamination': 0,
        'Terminated Bacterial Contamination,No Growth': 0,
        'Terminated Bacterial Contamination,Fungus Contamination,Sample Quality': 0,
        'Terminated Fibroblast Overgrowth': 0,
        'Terminated Feeder Contamination,No Growth': 0,
        'Verified Normal nan': 0,
        'Verified Normal Fibroblast Overgrowth': 0,
        'Verified Tumor nan': 1,
        'Verified Tumor Stopped Growing': 0,
        'Terminated Feeder Contamination': 0,
        'Verified Tumor Fingerprinting Failure': 2,
        'Verified Tumor Unlisted': 0,
        'Verified Normal Mouse Contamination': 0,
        'Verified Normal Stopped Growing': 0,
        'Verified Normal Unlisted': 0,
        'Verified Normal Sample Purity': 0,
        'Terminated Fingerprinting Failure': 2,
        'Terminated Bacterial Contamination,Fungus Contamination': 0,
        'Terminated Bacterial Contamination,Fungus Contamination,No Growth': 0,
        'Terminated No Growth,Sample Quality': 0,
        'Terminated Sample Quality': 0,
        'Terminated Lab Error,Sample Purity': 0,
        'Terminated Mouse Contamination': 0,
        'Terminated Fibroblast Overgrowth,Mouse Contamination': 0,
        'Verified Tumor No Growth': 0,
        'Terminated No Growth After Thaw': 0,
        'Terminated No Growth,Sample Purity,Sample Quality': 0,
        'Terminated Lab Error': 0,
        'Terminated Fingerprinting Failure,Lab Error': 0,
        'Verified Normal No Growth': 0,
        'Terminated No Growth,Sample Purity': 0,
        'Terminated Sample Purity': 0,
        'Verified Tumor Terminated': 0,
        'Terminated Terminated': 0,
        'Terminated Received/In Container': 0,
        'Completed Terminated': 0,
        'Completed Received/Out of Storage': 0,
        'Terminated nan': 0,
        'nan nan': 2,
        'Verified Tumor Received/Out of Storage': 2,
        'Verified Tumor Received/In Container': 2,
        'Verified Normal Terminated': 0,
        'Verified Normal Received/In Container': 2,
        'Verified Normal Received/Out of Storage': 2,
        'nan Terminated': 0,
        'nan Received/In Container': 2,
        'Terminated Received/Out of Storage': 0,
        'nan Received/Out of Storage': 2,
        'Completed nan': 1,
    }
    
    # Checking resolution and mapping back
    x["Fail Mode"] = [str(x) for x in x["Fail Mode"]]
    
    series = x["Resolution_modified"] + " " + x["Fail Mode"]
    series = [str(x) for x in series]

    x["Growth"] = series
    x = x.replace({"Growth": mapping_dict})

    x = x[x["Growth"] != 2]
    return x

In [15]:
data = growth_check(data)

In [16]:
def create_score(x):
    Lookup_table = {
        "PRIMARY":19,
        "P0":18,
        "P1":17,
        "P2":16,
        "P3":15,
        "P4":14,
        "P5":13,
        "P6":12,
        "P7":11,
        "P8":10,
        "P9":9,
        "P10":8,
        "P11":7,
        "P12":6,
        "P13":5,
        "P14":4,
        "P15":3,
        "P16":2,
        "P17":1,
    }

    test = [x.upper().split("_") for x in x["external_id_validation"]]
    
    value = []
    for t in test:
        for key in t:
            if key in Lookup_table:
                value_tmp = Lookup_table[key]
                break
            else:
                value_tmp = 0

        value.append(value_tmp)
        
    x["test"] = value
    
    return x

In [17]:
empty_numpy_panel = np.empty([1,len(list(Panel)[5:])])
empty_numpy_panel.fill(np.nan)
empty_frame_panel = pd.DataFrame(empty_numpy_panel, columns=list(Panel)[5:])

empty_numpy_RNA = np.empty([1,len(list(RNA)[4:])])
empty_numpy_RNA.fill(np.nan)
empty_frame_RNA = pd.DataFrame(empty_numpy_RNA, columns=list(RNA)[4:])

empty_numpy_WES = np.empty([1,len(list(WES)[5:])])
empty_numpy_WES.fill(np.nan)
empty_frame_WES = pd.DataFrame(empty_numpy_WES, columns=list(WES)[5:])

In [18]:
def add_nan_panel(data_tmp, empty_numpy_panel):
    data_tmp = pd.concat([data_tmp.reset_index(drop=True), pd.concat([empty_frame_panel]*len(data_tmp)).reset_index(drop=True)], axis=1)
    return(data_tmp)

def add_nan_RNA(data_tmp, empty_frame_RNA):
    data_tmp = pd.concat([data_tmp.reset_index(drop=True), pd.concat([empty_frame_RNA]*len(data_tmp)).reset_index(drop=True)], axis=1)
    return(data_tmp)

def add_nan_WES(data_tmp, empty_frame_WES):
    data_tmp = pd.concat([data_tmp.reset_index(drop=True), pd.concat([empty_frame_WES]*len(data_tmp)).reset_index(drop=True)], axis=1)
    return(data_tmp)

In [19]:
spreadsheet = []

for i, cell_line in enumerate(data["Cell Line"].unique()):
    data_tmp = data[data["Cell Line"] == cell_line]
    # At least one that isn't Verified Normal
    if all(data_tmp["Resolution_modified"] == "Verified Normal") != True:
        # First get the right data from Panel/RNA
        if cell_line.endswith("T"):
            Panel_tmp = Panel[Panel["participant"] == cell_line[:-1]]
            RNA_tmp = RNA[RNA["participant"] == cell_line[:-1]]
            WES_tmp = WES[WES["participant"] == cell_line[:-1]]
        else:
            Panel_tmp = Panel[Panel["participant"] == cell_line]
            RNA_tmp = RNA[RNA["participant"] == cell_line]
            WES_tmp = WES[WES["participant"] == cell_line]

        #Add Panel data
        if not Panel_tmp.empty:
            # Check if all are not VN
            if all(Panel_tmp["PANEL_tn_decision_clean"] == "VN") != True:
                # Check if multiple sequencing results for cell line
                if len(Panel_tmp) == 1:
                    Panel_tmp = Panel_tmp.iloc[0:, 5:]
                    data_tmp = pd.concat([data_tmp.reset_index(drop=True), pd.concat([Panel_tmp]*len(data_tmp)).reset_index(drop=True)], axis=1)
                else:
                    # Loop through these panels
                    Panel_tmp = create_score(Panel_tmp)
                    Panel_tmp = Panel_tmp[Panel_tmp["test"] == max(Panel_tmp["test"])]
                    if all(Panel_tmp["PANEL_tn_decision_clean"] == "VN") != True:
                        if len(Panel_tmp) == 1:
                            Panel_tmp = Panel_tmp.iloc[0:1, 5:-1]
                            data_tmp = pd.concat([data_tmp.reset_index(drop=True), pd.concat([Panel_tmp]*len(data_tmp)).reset_index(drop=True)], axis=1)
                        else:
                            # Just picking the one which is not VN
                            Panel_tmp = Panel_tmp[Panel_tmp["PANEL_tn_decision_clean"] != "VN"]
                            if not Panel_tmp.empty:
                                Panel_tmp_tmp = Panel_tmp[Panel_tmp["PANEL_tn_decision_clean"] == "VT"]
                                if not Panel_tmp_tmp.empty:
                                    Panel_tmp_tmp = Panel_tmp_tmp.iloc[0:1, 5:-1]
                                    data_tmp = pd.concat([data_tmp.reset_index(drop=True), pd.concat([Panel_tmp_tmp]*len(data_tmp)).reset_index(drop=True)], axis=1)
                                else:
                                    Panel_tmp = Panel_tmp.iloc[0:1, 5:-1]
                                    data_tmp = pd.concat([data_tmp.reset_index(drop=True), pd.concat([Panel_tmp]*len(data_tmp)).reset_index(drop=True)], axis=1)
                            else:
                                #Add empty columns
                                data_tmp = add_nan_panel(data_tmp, empty_frame_panel)
                                
                    else:
                        #Add empty columns
                        data_tmp = add_nan_panel(data_tmp, empty_frame_panel)

            else:
                #Add empty columns
                data_tmp = add_nan_panel(data_tmp, empty_frame_panel)
        
        else:
            #Add empty columns
            data_tmp = add_nan_panel(data_tmp, empty_frame_panel)
                
        #Add RNA data
        if not RNA_tmp.empty:
            if len(RNA_tmp) == 1:
                RNA_tmp = RNA_tmp.iloc[0:1, 4:]
                data_tmp = pd.concat([data_tmp.reset_index(drop=True), pd.concat([RNA_tmp]*len(data_tmp)).reset_index(drop=True)], axis=1)
            else:
                if len(data_tmp) == 1:
                    RNA_tmp = RNA_tmp.iloc[0:1, 4:]
                    data_tmp = pd.concat([pd.concat([data_tmp]*len(RNA_tmp)).reset_index(drop=True), RNA_tmp.reset_index(drop=True)], axis=1)
                else:
                    # Just pick one
                    RNA_tmp = RNA_tmp.iloc[0:1, 4:]
                    data_tmp = pd.concat([data_tmp.reset_index(drop=True), pd.concat([RNA_tmp]*len(data_tmp)).reset_index(drop=True)], axis=1)
                        
        else:
            data_tmp = add_nan_RNA(data_tmp, empty_frame_RNA)
                          
        #Add WES data
        if not WES_tmp.empty:
            if len(WES_tmp) == 1:
                WES_tmp = WES_tmp.iloc[0:1, 5:]
                data_tmp = pd.concat([data_tmp.reset_index(drop=True), pd.concat([WES_tmp]*len(data_tmp)).reset_index(drop=True)], axis=1)
            else:
                if len(data_tmp) == 1:
                    WES_tmp = WES_tmp.iloc[0:1, 5:]
                    data_tmp = pd.concat([pd.concat([data_tmp]*len(WES_tmp)).reset_index(drop=True), WES_tmp.reset_index(drop=True)], axis=1)
                else:
                    # Just pick one
                    WES_tmp = WES_tmp.iloc[0:1, 5:]
                    data_tmp = pd.concat([data_tmp.reset_index(drop=True), pd.concat([WES_tmp]*len(data_tmp)).reset_index(drop=True)], axis=1)
                        
        else:
            data_tmp = add_nan_WES(data_tmp, empty_frame_WES)
        
    else:
        #Add empty columns
        data_tmp = add_nan_panel(data_tmp, empty_frame_panel)
        data_tmp = add_nan_RNA(data_tmp, empty_frame_RNA)
        data_tmp = add_nan_WES(data_tmp, empty_frame_WES)
        
    data_tmp["RNA_match"] = "Unsure"
    spreadsheet.append(data_tmp)

<ipython-input-16-c83dd44c1f94>:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x["test"] = value


In [20]:
spreadsheet = pd.concat(spreadsheet)

In [47]:
#renaming header
spreadsheet.columns = ["_".join(x.split(" ")) for x in spreadsheet.columns]
spreadsheet = spreadsheet.rename(columns={"Resolution":"Resolution_Original", "Resolution_modified":"Resolution", "T/N":"Type"})

In [48]:
# Create small spreadsheet if both no RNA and Panel data
spreadsheet["PANEL_renamed_bai_file"] = [str(x) for x in spreadsheet["PANEL_renamed_bai_file"]]
spreadsheet["RNA_fastq1"] = [str(x) for x in spreadsheet["RNA_fastq1"]]
spreadsheet_small = spreadsheet[(spreadsheet["PANEL_renamed_bai_file"] != "nan") | (spreadsheet["RNA_fastq1"] != "nan")]
new_index = [spreadsheet_small.iloc[i]["Cell_Line_Starting_Sample"] + "_" + str(x) for i,x in enumerate(spreadsheet_small.index)]
spreadsheet_small.index = new_index

In [49]:
spreadsheet_small.to_csv(data_location / "spreadsheet_small.tsv", encoding = 'utf-8', sep='\t')
spreadsheet.to_csv(data_location / "spreadsheet_compleet.tsv", encoding = 'utf-8', sep='\t')

## RNA/WES data for Gwen

In [1579]:
Gwen_meta_data = data

In [1580]:
Gwen_meta_data["participant"] = [x[:-1] if x.endswith("T") else x for x in data["Cell Line"]]
Gwen_meta_data = Gwen_meta_data[["Primary Disease", "Tissue Site", "Tumor Type", "Starting Material Type", "Dimension", "participant"]]
Gwen_meta_data = Gwen_meta_data.drop_duplicates(keep=False)

In [1586]:
Gwen_RNA = pd.merge(Gwen_meta_data, RNA_meta, how='inner', on='participant')

In [1587]:
Gwen_RNA

,Primary Disease,Tissue Site,Tumor Type,Starting Material Type,Dimension,participant,entity:sample_id,aggregation_product_name_rna,bai_file_rna,bam_file_rna,...,fig_random,individual_alias,rnaseqc2_exon_counts,rnaseqc2_gene_counts,rnaseqc2_gene_tpm,rnaseqc2_insertsize_distr,rnaseqc2_metrics,sample_PDO,sequencing_technology,external_rna_id
0,NaN,NaN,NaN,Fresh Tissue,NaN,SP025,SP025-Tumor-SM-C1JOO,Tru-Seq Strand Specific Large Insert RNA Seque...,gs://fc-020eb013-7449-4da7-8c7f-98294243830a/S...,gs://fc-020eb013-7449-4da7-8c7f-98294243830a/S...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,Fresh Tissue,NaN,SP025,SP025-Tumor-SM-HPY1Q,NaN,gs://fc-1d1574d1-5469-4d1d-b11d-ea13b2f0d5d9/C...,gs://fc-1d1574d1-5469-4d1d-b11d-ea13b2f0d5d9/C...,...,NaN,SP025,gs://fc-b3a85406-97fd-452d-bbff-a44c92749ada/f...,gs://fc-b3a85406-97fd-452d-bbff-a44c92749ada/f...,gs://fc-b3a85406-97fd-452d-bbff-a44c92749ada/f...,gs://fc-b3a85406-97fd-452d-bbff-a44c92749ada/f...,gs://fc-b3a85406-97fd-452d-bbff-a44c92749ada/f...,PDO-16164,ICE,NaN
2,t-pll,NaN,NaN,Whole Blood,NaN,DW036,RP-571_DW036T_v2_RNA_OnPrem,NaN,gs://fc-1d1574d1-5469-4d1d-b11d-ea13b2f0d5d9/C...,gs://fc-1d1574d1-5469-4d1d-b11d-ea13b2f0d5d9/C...,...,NaN,DW036,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DW036T
3,melanoma,NaN,NaN,Fresh Tissue,NaN,CY001,CY001-Tumor-SM-B288N,Tru-Seq Strand Specific Large Insert RNA Seque...,gs://fc-020eb013-7449-4da7-8c7f-98294243830a/C...,gs://fc-020eb013-7449-4da7-8c7f-98294243830a/C...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,anaplastic thyroid cancer,NaN,NaN,Pleural Effusion,NaN,JL16,CCLF_Mix_RNASeq_June2016-JL16-Tumor-SM-5LV2J,Tru-Seq Strand Specific Large Insert RNA Seque...,gs://fc-020eb013-7449-4da7-8c7f-98294243830a/C...,gs://fc-020eb013-7449-4da7-8c7f-98294243830a/C...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,esophageal carcinoma,Esophagus,Primary,Fresh Tissue,3D,CCLF_AB1017,RP-571_CCLF_AB1017T_v1_RNA_OnPrem,NaN,gs://fc-1d1574d1-5469-4d1d-b11d-ea13b2f0d5d9/C...,gs://fc-1d1574d1-5469-4d1d-b11d-ea13b2f0d5d9/C...,...,NaN,CCLF_AB1017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CCLF_AB1017T
96,stomach adenocarcinoma,Stomach,Primary,Fresh Tissue,3D,CCLF_AB1030,RP-571_CCLF_AB1030T_v1_RNA_OnPrem,NaN,gs://fc-1d1574d1-5469-4d1d-b11d-ea13b2f0d5d9/C...,gs://fc-1d1574d1-5469-4d1d-b11d-ea13b2f0d5d9/C...,...,NaN,CCLF_AB1030,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CCLF_AB1030T
97,alveolar rhabdomyosarcoma,Leg,Primary,Fresh Tissue,2D,CCLF_PEDS1064,RP-571_CCLF_PEDS1064T_v1_RNA_OnPrem,NaN,gs://fc-bdfe4c68-a2bc-41b0-ac15-931e84fb1092/C...,gs://fc-bdfe4c68-a2bc-41b0-ac15-931e84fb1092/C...,...,NaN,CCLF_PEDS1064,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CCLF_PEDS1064T
98,squamous cell carcinoma,Head and neck region,Primary,PDX Tissue,2D,HG002,HG002-Tumor-SM-HPY1X,NaN,gs://fc-1d1574d1-5469-4d1d-b11d-ea13b2f0d5d9/C...,gs://fc-1d1574d1-5469-4d1d-b11d-ea13b2f0d5d9/C...,...,NaN,HG002,gs://fc-b3a85406-97fd-452d-bbff-a44c92749ada/f...,gs://fc-b3a85406-97fd-452d-bbff-a44c92749ada/f...,gs://fc-b3a85406-97fd-452d-bbff-a44c92749ada/f...,gs://fc-b3a85406-97fd-452d-bbff-a44c92749ada/f...,gs://fc-b3a85406-97fd-452d-bbff-a44c92749ada/f...,PDO-16164,ICE,NaN
